In [9]:
using JuMP, Gurobi, Random

In [19]:
using JuMP, Gurobi, StatsBase, DataFrames, CSV, LinearAlgebra, Distributions, Random

In [57]:
using Dualization

┌ Info: Precompiling Dualization [191a621a-6537-11e9-281d-650236a99e60]
└ @ Base loading.jl:1278


In [56]:
import Pkg; Pkg.add("Dualization")

   Updating registry at `~/.julia/registries/General`
######################################################################### 100.0%
  Resolving package versions...
  Installed Dualization ─ v0.3.4
Updating `~/.julia/environments/v1.5/Project.toml`
  [191a621a] + Dualization v0.3.4
Updating `~/.julia/environments/v1.5/Manifest.toml`
  [191a621a] + Dualization v0.3.4


In [306]:
function inner_primal_infinity(X, y, β0, epsilon, delta)
    
    #Non-adaptive version, that optimizes wrt CVaR, with OLS, and lasso penalty
    #alpha: Risk value
    
    n, p = size(X)

    # Create model
    model = Model(with_optimizer(Gurobi.Optimizer))
    set_optimizer_attribute(model, "OutputFlag", 0)

    # Add variables
    @variable(model, β[i=1:n,j=1:p]) 
    @variable(model, b[i=1:n]>=0) 

    # Add objective
    @objective(model, Min, sum(b[i] for i=1:n))
    
    #@constraint(model,[i=1:n], y .- dot(X,β) .<= b)
    #@constraint(model,[i=1:n],-y .+ dot(X,β) .<= b)
    @constraint(model, res_plus[i=1:n],  - y[i] + dot(X[i,:],β[i,:]) <= b[i])
    @constraint(model, res_minus[i=1:n],  y[i] - dot(X[i,:],β[i,:]) <= b[i])

    @constraint(model, diff_plus[i=2:n],   β[i,:] .- β[i-1,:] .<= delta)
    @constraint(model, diff_minus[i=2:n], - β[i,:] .+ β[i-1,:] .<= delta)
    
    @constraint(model, diff_0_plus[i=1:n],   β[i,:] .- β0 .<= epsilon)
    @constraint(model, diff_0_minus[i=1:n], - β[i,:] .+ β0 .<= epsilon)
    
    optimize!(model);
    
    return model, getvalue.(β)
end

inner_primal_infinity (generic function with 1 method)

In [277]:
Z = get_Z(X)
λ = Random.rand(n)

10-element Array{Float64,1}:
 0.791563863554702
 0.5796189138571763
 0.9100579881453945
 0.5056536604729605
 0.7719481674585646
 0.7102191914978586
 0.0965590441026678
 0.7283500723140499
 0.22105848314216003
 0.6861513606223042

In [279]:
Z

10×30 Array{Float64,2}:
 0.231457  0.99787  0.236804  0.0       …  0.0       0.0       0.0
 0.0       0.0      0.0       0.935695     0.0       0.0       0.0
 0.0       0.0      0.0       0.0          0.0       0.0       0.0
 0.0       0.0      0.0       0.0          0.0       0.0       0.0
 0.0       0.0      0.0       0.0          0.0       0.0       0.0
 0.0       0.0      0.0       0.0       …  0.0       0.0       0.0
 0.0       0.0      0.0       0.0          0.0       0.0       0.0
 0.0       0.0      0.0       0.0          0.0       0.0       0.0
 0.0       0.0      0.0       0.0          0.0       0.0       0.0
 0.0       0.0      0.0       0.0          0.191178  0.136631  0.889505

In [281]:
transpose(λ)*Z

1×30 Transpose{Float64,Array{Float64,1}}:
 0.183213  0.789878  0.187445  0.542346  …  0.131177  0.0937494  0.610335

In [343]:
function inner_dual_infinity_shuvo(X, y, β0, epsilon, delta)
    
    #Non-adaptive version, that optimizes wrt CVaR, with OLS, and lasso penalty
    #alpha: Risk value
    
    T, p = size(X)
    Z = get_Z(X)
    
    # Create model
    model = Model(with_optimizer(Gurobi.Optimizer))
    set_optimizer_attribute(model, "OutputFlag", 0)
    
    # Add variables
    @variable(model, λ[i=1:2, j=1:T] >= 0) 
    @variable(model, ν[i=1:2, j=1:T-1, k=1:p]>=0) 
    @variable(model, μ[i=1:2, j=1:T, k=1:p]>=0)

    # Add objective
    @objective(model, Max, dot(λ[1,:],y) - dot(λ[2,:],y) 
                            - delta * sum(sum(ν[1,t,i]+ν[2,t,i] for i=1:p) for t=1:T-1)
                            - sum(dot(epsilon .+ β0, μ[1,t,:]) for t = 1:T)
                            + sum(dot(epsilon .- β0, μ[2,t,:]) for t = 1:T))
    
    @constraint(model,[t=1:T], λ[1,:] .+ λ[2,:] .== 1)
        
        
    @constraint(model, transpose(λ[2,:])*Z-transpose(λ[1,:])*Z 
                        + sum(transpose(ν[1,t,:])*(get_A(X, t+1).-get_A(X, t)) for t=1:T-1)
                        + sum(transpose(ν[2,t,:])*(-get_A(X, t+1).+get_A(X, t)) for t=1:T-1)
                        + sum(transpose(μ[1,t,:])*get_A(X,t) for t=1:T)
                        - sum(transpose(μ[2,t,:])*get_A(X,t) for t=1:T) .== 0)
    
    
    optimize!(model);
    
    return model
end

inner_dual_infinity_shuvo (generic function with 1 method)

In [344]:
n = 30
p = 6
X = Random.rand(n,p)
#X[:,-1] = np.ones(n)
y = Random.rand(n)
beta_0 = Random.rand(p)
epsilon = 0.1
delta = 0.1
#beta = inner_dual_infinity_leo_original_signs(X,y,beta_0,epsilon,delta)
#beta = inner_dual_infinity_leo(X,y,beta_0,epsilon,delta)
model = inner_dual_infinity_shuvo(X,y,beta_0,epsilon,delta)
print("Obj value Dual:", objective_value(model))

#model = inner_dual_infinity_leo(X,y,beta_0,epsilon,delta)
#print("Obj value Dual Leo:", objective_value(model))

m, beta = inner_primal_infinity(X,y,beta_0,epsilon,delta)
print("Obj value Primal:", objective_value(model))

#print("Primal 1", value(m.obj))

Academic license - for non-commercial use only - expires 2022-07-20
Obj value Dual:0.08329960556174407Academic license - for non-commercial use only - expires 2022-07-20
Obj value Primal:0.08329960556174407

In [302]:
objective_value(model)

0.5917642668061699

In [249]:
e = zeros((1,10*2))
e[5:10] = Random.rand((6))
e

1×20 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.460026  0.24066  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [250]:
function get_Y(X, t)
    T, p = size(X)
    Y = zeros(1,T*p)
    Y[(t-1)*p+1:t*p] = X[t,:]
    return Y
end

get_Y (generic function with 1 method)

In [255]:
function get_Z(X)
    T, p = size(X)
    Z = zeros(T, T*p)
    for t=1:T
        Z[t,:] = get_Y(X,t)
    end
    return Z
end

get_Z (generic function with 1 method)

In [271]:
get_A(X,1).-get_A(X,2)

3×30 Array{Float64,2}:
 1.0  0.0  0.0  -1.0   0.0   0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0   0.0  -1.0   0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0   0.0   0.0  -1.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [266]:
1 * Matrix(I, p, p)

3×3 Array{Int64,2}:
 1  0  0
 0  1  0
 0  0  1

In [269]:
function get_A(X, t)
    T, p = size(X)
    A = zeros(p,T*p)
    A[:,(t-1)*p+1:t*p] = 1 * Matrix(I, p, p)
    return A
end

get_A (generic function with 1 method)

In [268]:
get_A(X,1)

3×30 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [313]:
function inner_dual_infinity_leo(X, y, β0, epsilon, delta)
    
    #Non-adaptive version, that optimizes wrt CVaR, with OLS, and lasso penalty
    #alpha: Risk value
    
    T, p = size(X)

    # Create model
    model = Model(with_optimizer(Gurobi.Optimizer))
    set_optimizer_attribute(model, "OutputFlag", 0)
    
    # Add variables
    @variable(model, λ[i=1:2,j=1:T]>=0) 
    @variable(model, ν[i=1:2, j=1:T,k=1:p]>=0) 
    @variable(model, μ[i=1:2, j=1:T,k=1:p]>=0)

    # Add objective
    @objective(model, Max, dot(λ[1,:],y) - dot(λ[2,:],y) 
                            - delta * sum(sum(ν[1,t,i]+ν[2,t,i] for i=1:p) for t=1:T)
                            - sum(dot(epsilon .+ β0, μ[1,t,:]) for t=1:T)
                            + sum(dot(epsilon .- β0, μ[2,t,:]) for t=1:T)
        )
    
    @constraint(model,[t=1:T], λ[1,:] .+ λ[2,:] .== 1)
    @constraint(model,[t=1:T-1], X[t,:]*(λ[1,t]-λ[2,t]) .+ μ[2,t,:]-μ[1,t,:]
                                            .+ ν[2,t,:]-ν[1,t,:]
                                            .+ ν[2,t+1,:]-ν[1,t+1,:] .== 0)
    @constraint(model, X[T,:]*(λ[1,T]-λ[2,T]) .+ μ[2,T,:]-μ[1,T,:]
                                            .+ ν[2,T,:]-ν[1,T,:] .== 0)

    @constraint(model, [i=1:p], ν[1,1,i] == 0)
    @constraint(model, [i=1:p], ν[2,1,i] == 0)
    
    optimize!(model);
    
    return getvalue.(λ)
end

inner_dual_infinity_leo (generic function with 1 method)

In [199]:
function inner_dual_infinity_leo_original_signs(X, y, β0, epsilon, delta)
    
    #Non-adaptive version, that optimizes wrt CVaR, with OLS, and lasso penalty
    #alpha: Risk value
    
    T, p = size(X)

    # Create model
    model = Model(with_optimizer(Gurobi.Optimizer))
    #set_optimizer_attribute(model, "OutputFlag", 0)
    
    # Add variables
    @variable(model, λ[i=1:2,j=1:T]<=0) 
    @variable(model, ν[i=1:2, j=1:T,k=1:p]<=0) 
    @variable(model, μ[i=1:2, j=1:T,k=1:p]<=0)

    # Add objective
    @objective(model, Max, dot(λ[2,:],y) - dot(λ[1,:],y) 
                            + delta * sum(sum(ν[1,t,i]+ν[2,t,i] for i=1:p) for t=1:T)
                            + sum(dot(epsilon .+ β0, μ[1,t,:]) for t=1:T)
                            + sum(dot(epsilon .- β0, μ[2,t,:]) for t=1:T)
        )
    
    @constraint(model,[t=1:T], -λ[1,:] .- λ[2,:] .== 1)
    @constraint(model,[t=1:T-1], X[t,:]*(λ[2,t]-λ[1,t]) .+ μ[1,t,:]-μ[2,t,:]
                                            .+ ν[1,t,:]-ν[2,t,:]
                                            .+ ν[1,t+1,:]-ν[2,t+1,:] .== 0)
    @constraint(model, X[T,:]*(λ[2,T]-λ[1,T]) .+ μ[1,T,:]-μ[2,T,:]
                                            .+ ν[1,T,:]-ν[2,T,:] .== 0)

    @constraint(model, [i=1:p], ν[1,1,i] == 0)
    @constraint(model, [i=1:p], ν[2,1,i] == 0)
    
    optimize!(model);
    
    return getvalue.(λ)
end

inner_dual_infinity_leo_original_signs (generic function with 1 method)

In [97]:
n = 1
p = 1
X = Random.rand(n,p)
#X[:,-1] = np.ones(n)
y = Random.rand(n)
beta_0 = Random.rand(p)
epsilon = 0.1
delta = 0.01
m, beta = inner_primal_infinity(X,y,beta_0,epsilon,delta)
#print("Primal 1", value(m.obj))

Academic license - for non-commercial use only - expires 2022-07-20
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 4 rows, 2 columns and 6 nonzeros
Model fingerprint: 0xf1865ffb
Coefficient statistics:
  Matrix range     [6e-02, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-01, 6e-01]
Presolve removed 4 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.0374523e-01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  6.037452260e-01

User-callback calls 25, time in user-callback 0.00 sec


(A JuMP Model
Minimization problem with:
Variables: 2
Objective function type: VariableRef
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 4 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Gurobi
Names registered in the model: b, diff_0_minus, diff_0_plus, diff_minus, diff_plus, res_minus, res_plus, β, [0.3519194308823449])

In [98]:
m

A JuMP Model
Minimization problem with:
Variables: 2
Objective function type: VariableRef
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 4 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Gurobi
Names registered in the model: b, diff_0_minus, diff_0_plus, diff_minus, diff_plus, res_minus, res_plus, β

In [83]:
function inner_primal_infinity(X, y, β0, epsilon, delta)
    
    #Non-adaptive version, that optimizes wrt CVaR, with OLS, and lasso penalty
    #alpha: Risk value
    
    n, p = size(X)

    # Create model
    model = Model(with_optimizer(Gurobi.Optimizer))

    # Add variables
    @variable(model, a[i=1:n,j=1:p]) 
    @variable(model, b[i=1:n]>=0) 

    # Add objective
    @objective(model, Min, sum(b[i] for i=1:n))
    
    #@constraint(model,[i=1:n], y .- dot(X,β) .<= b)
    #@constraint(model,[i=1:n],-y .+ dot(X,β) .<= b)
    @constraint(model,[i=1:n],- y[i] + dot(X[i,:],a[i,:]) <= b[i])
    @constraint(model,[i=1:n],  y[i] - dot(X[i,:],a[i,:]) <= b[i])

    @constraint(model,[i=2:n],   a[i,:] .- a[i-1,:] .<= delta)
    @constraint(model,[i=2:n], - a[i,:] .+ a[i-1,:] .<= delta)
    
    @constraint(model,[i=1:n],   a[i,:] .- β0 .<= epsilon)
    @constraint(model,[i=1:n], - a[i,:] .+ β0 .<= epsilon)
    
    optimize!(model);
    
    return model, getvalue.(a)
end

inner_primal_infinity (generic function with 1 method)

In [125]:
beta_0

2-element Array{Float64,1}:
 13.0
 14.0

In [133]:
Random.rand(n,p)

3×2 Array{Float64,2}:
 0.0863508  0.288636
 0.766061   0.586348
 0.871986   0.936701

In [146]:
X

3×2 Array{Float64,2}:
 1.0  4.0
 2.0  5.0
 3.0  6.0

In [145]:
n = 3
p = 2
X = Array([[1., 2., 3.] [4., 5., 6.]])#Random.rand(n,p)
#X[:,-1] = np.ones(n)
y = Array([10.,11.,12.])#Random.rand(n)
beta_0 = Array([13.,14.])#Random.rand(p)
epsilon = 0.1
delta = 0.01
m, beta = inner_primal_infinity(X,y,beta_0,epsilon,delta)
#print("Primal 1", value(m.obj))

Academic license - for non-commercial use only - expires 2022-07-20
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 26 rows, 9 columns and 46 nonzeros
Model fingerprint: 0xac5b704b
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-02, 1e+01]
Presolve removed 26 rows and 9 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.5290000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  2.529000000e+02

User-callback calls 28, time in user-callback 0.00 sec


(A JuMP Model
Minimization problem with:
Variables: 9
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 26 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 3 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Gurobi
Names registered in the model: b, diff_0_minus, diff_0_plus, diff_minus, diff_plus, res_minus, res_plus, β, [12.9 13.9; 12.9 13.9; 12.9 13.9])

In [147]:
dual_model = dualize(m)

A JuMP Model
Maximization problem with:
Variables: 29
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 9 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 26 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [32]:
beta = inner_primal_infinity(X,y,beta_0,epsilon,delta)

Academic license - for non-commercial use only - expires 2022-07-20
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 210 rows, 60 columns and 400 nonzeros
Model fingerprint: 0xc48ab349
Coefficient statistics:
  Matrix range     [2e-03, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-02, 8e-01]
Presolve removed 210 rows and 60 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.2475641e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  5.247564085e+00

User-callback calls 31, time in user-callback 0.00 sec


10×5 Array{Float64,2}:
 0.0525981  0.584418  0.534952  0.231315  0.466786
 0.0525981  0.584418  0.534952  0.231315  0.466786
 0.0525981  0.584418  0.534952  0.231315  0.466786
 0.0525981  0.584418  0.534952  0.231315  0.466786
 0.0525981  0.584418  0.534952  0.231315  0.466786
 0.0525981  0.584418  0.534952  0.231315  0.466786
 0.0525981  0.584418  0.534952  0.231315  0.466786
 0.0525981  0.584418  0.534952  0.231315  0.466786
 0.0525981  0.584418  0.534952  0.231315  0.466786
 0.0525981  0.584418  0.534952  0.231315  0.466786

In [ ]:
function l1_regression(X, y, rho)
    # number of observations
    n = nrow(X)
    # number of estimators
    m = ncol(X)
    
    # big M constant. Note big M should be small for l0 to work properly
    M = 25
    
    # convert input data frames to matrix/vector
    y_vector = convert(Matrix, y[:,:])
    X_matrix = convert(Matrix, X[:,:])
    
    # build model
    model = Model(with_optimizer(Gurobi.Optimizer))
    ##### Load Packagesset_optimizer_attribute(model, "OutputFlag", 0)
    # insert variables and constraints
    @variable(model,beta[i=1:m])
    @variable(model,z[i=1:m]>=0)
    @variable(model,u[i=1:n]>=0)
    @variable(model, sse>=0)
    @constraint(model,[i=1:m], beta[i]<=z[i])
    @constraint(model,[i=1:m], beta[i]>=-z[i])
    @constraint(model,[i=1:n],y_vector[i]-dot(X_matrix[i,:],beta)<=u[i])
    @constraint(model,[i=1:n],-y_vector[i]+dot(X_matrix[i,:],beta)<=u[i])
    @constraint(model, sum(u) <= sse)
    @objective(model,Min, sse + rho*sum(z[i] for i=1:m))
    
    # optimize
    optimize!(model);
    
    z_here = getvalue.(z)
    b_here = getvalue.(beta)
    # return estimated betas
    return (getvalue.(beta))
end